# AI 語音降噪 Colab
這個 Colab 筆記本會使用 [SpeechBrain](https://github.com/speechbrain/speechbrain) 的 MetricGAN+ 模型，使用 GPU 進行語音降噪。

載入 Google Drive 中的影音或聲音檔案，並將結果產生為聲音檔 (.wav)。

In [ ]:
#@title 🎉 一鍵啟動語音降噪 / One-click Audio Denoiser
print("📂 掛載雲端硬碟中 / Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive')

input_path = "/content/drive/MyDrive/input.wav"  #@param {type:"string"}
use_gpu = True  #@param {type:"boolean"}

print("📦 正在安裝套件 / Installing packages...")
!pip install -q speechbrain torchaudio ffmpeg-python
print("✅ 套件安裝完成 / Packages installed!")

import os, glob, subprocess, torch, torchaudio
from speechbrain.pretrained.interfaces import SpectralMaskEnhancement

video_exts = ['.mp4','.mov','.avi','.mkv','.flv','.webm']
audio_exts = ['.wav','.mp3','.flac','.ogg','.m4a']

def extract_audio(src, dst):
    subprocess.run(['ffmpeg','-y','-i', src, '-vn','-ac','1', dst], check=True)

def denoise_file(src, enhancer):
    tmp_wav = '/content/tmp_input.wav'
    ext = os.path.splitext(src)[1].lower()
    if ext in video_exts:
        extract_audio(src, tmp_wav)
        processed_input = tmp_wav
    else:
        processed_input = src
    enhanced = enhancer.enhance_file(processed_input)
    out_file = os.path.splitext(src)[0] + '_denoised.wav'
    torchaudio.save(out_file, enhanced.squeeze(0), 16000)
    if processed_input == tmp_wav and os.path.exists(tmp_wav):
        os.remove(tmp_wav)
    print(f"🎵 已處理 / Done: {out_file}")

device = 'cuda' if (use_gpu and torch.cuda.is_available()) else 'cpu'
print(f"💻 使用裝置 / Device: {device}")
enhancer = SpectralMaskEnhancement.from_hparams(
    source='speechbrain/metricgan-plus-voicebank',
    savedir='pretrained_models/metricgan-plus-voicebank',
    run_opts={'device': device}
)

if os.path.isdir(input_path):
    print('📁 偵測到資料夾，開始批次處理 / Folder detected, processing all files...')
    files = [f for f in glob.glob(os.path.join(input_path, '*')) if os.path.splitext(f)[1].lower() in video_exts + audio_exts]
    for f in sorted(files):
        denoise_file(f, enhancer)
else:
    denoise_file(input_path, enhancer)

print('🎉 全部搞定 / All done!')
